In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split     # import module for train test split
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import evaluate_print
from scipy import stats

Using TensorFlow backend.


In [3]:
df=pd.read_csv("./Invisily_Access_Logs_Data_11Sep.csv")

In [4]:
#display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df.head()

,id,agent_id,client_physical_ip,client_pseudo_ip,service_id,service_physical_ip,service_pseudo_ip,client_tx_bytes,service_tx_bytes,difference_bytes,createdAt,updatedAt,log_start_time,log_end_time
0,26,7067,103.255.5.103,10.0.1.2,701,192.168.3.18,10.0.1.4,19347,52032,32685,2020-08-27 11:51:34,2020-08-27 11:51:34,1598508692,1598509007
1,32,7096,111.119.178.162,10.0.1.194,701,192.168.3.18,10.0.1.196,55508,137447,81939,2020-08-27 11:51:34,2020-08-27 11:51:34,1598509662,1598509776
2,48,7238,39.53.107.112,10.0.0.2,700,192.168.3.1,10.0.0.3,65615,505399,439784,2020-08-27 11:51:34,2020-08-27 11:51:34,1598507738,1598510917
3,74,7352,118.107.131.76,10.0.0.194,701,192.168.3.18,10.0.0.196,14768,77408,62640,2020-08-27 11:51:34,2020-08-27 11:51:34,1598510653,1598511031
4,82,7380,110.38.129.187,10.1.0.66,830,172.16.1.220,10.1.0.72,94493,116133,21640,2020-08-27 12:13:03,2020-08-27 12:13:03,1598508889,1598510156


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   2592 non-null   int64 
 1   agent_id             2592 non-null   int64 
 2   client_physical_ip   2592 non-null   object
 3   client_pseudo_ip     2592 non-null   object
 4   service_id           2592 non-null   int64 
 5   service_physical_ip  2592 non-null   object
 6   service_pseudo_ip    2592 non-null   object
 7   client_tx_bytes      2592 non-null   int64 
 8   service_tx_bytes     2592 non-null   int64 
 9   difference_bytes     2592 non-null   int64 
 10  createdAt            2592 non-null   object
 11  updatedAt            2592 non-null   object
 12  log_start_time       2592 non-null   int64 
 13  log_end_time         2592 non-null   int64 
dtypes: int64(8), object(6)
memory usage: 283.6+ KB


In [7]:
df.describe()

,id,agent_id,service_id,client_tx_bytes,service_tx_bytes,difference_bytes,log_start_time,log_end_time
count,2592.000000,2592.000000,2592.000000,2.592000e+03,2.592000e+03,2.592000e+03,2.592000e+03,2.592000e+03
mean,16168.442515,7269.949846,754.902006,6.068481e+05,7.776235e+06,7.715587e+06,1.599159e+09,1.599161e+09
std,9404.633129,110.213357,51.915657,3.680620e+06,6.454868e+07,6.329439e+07,3.973094e+05,3.972351e+05
min,26.000000,7008.000000,700.000000,4.000000e+01,4.000000e+01,0.000000e+00,1.598508e+09,1.598509e+09
25%,7660.500000,7195.000000,701.000000,1.059350e+04,2.446175e+04,1.233900e+04,1.598872e+09,1.598874e+09
50%,16302.500000,7297.000000,737.000000,3.280900e+04,1.187265e+05,6.285950e+04,1.599111e+09,1.599114e+09
75%,24262.500000,7367.000000,800.000000,1.511288e+05,6.662490e+05,5.406798e+05,1.599545e+09,1.599547e+09
max,32311.000000,7405.000000,841.000000,1.068441e+08,1.448077e+09,1.419875e+09,1.599833e+09,1.599833e+09


In [8]:
df.dtypes

id                      int64
agent_id                int64
client_physical_ip     object
client_pseudo_ip       object
service_id              int64
service_physical_ip    object
service_pseudo_ip      object
client_tx_bytes         int64
service_tx_bytes        int64
difference_bytes        int64
createdAt              object
updatedAt              object
log_start_time          int64
log_end_time            int64
dtype: object

In [9]:
df.groupby('agent_id').count()

,id,client_physical_ip,client_pseudo_ip,service_id,service_physical_ip,service_pseudo_ip,client_tx_bytes,service_tx_bytes,difference_bytes,createdAt,updatedAt,log_start_time,log_end_time
agent_id,,,,,,,,,,,,,
7008,8,8,8,8,8,8,8,8,8,8,8,8,8
7014,5,5,5,5,5,5,5,5,5,5,5,5,5
7026,23,23,23,23,23,23,23,23,23,23,23,23,23
7032,3,3,3,3,3,3,3,3,3,3,3,3,3
7039,18,18,18,18,18,18,18,18,18,18,18,18,18
7048,130,130,130,130,130,130,130,130,130,130,130,130,130
7056,1,1,1,1,1,1,1,1,1,1,1,1,1
7067,29,29,29,29,29,29,29,29,29,29,29,29,29
7072,5,5,5,5,5,5,5,5,5,5,5,5,5


In [10]:
def select_agent(df, agent_id):
    df_selected = df.loc[df['agent_id'] == agent_id]
    return df_selected

In [11]:
def show_unique_service_ids(df, agent_id):
    df_agent_selected = select_agent(df, agent_id)
    service_ids = df_agent_selected.groupby('service_id')['service_id'].first()
    print(service_ids)


In [12]:


def select_service(df,service_id):
    df_selected = df.loc[df['service_id'] == service_id]
    return df_selected



In [13]:


def select_features(df, feature_list):
    df_selected = df[feature_list]
    return df_selected



In [14]:
df.columns

Index(['id', 'agent_id', 'client_physical_ip', 'client_pseudo_ip',
       'service_id', 'service_physical_ip', 'service_pseudo_ip',
       'client_tx_bytes', 'service_tx_bytes', 'difference_bytes', 'createdAt',
       'updatedAt', 'log_start_time', 'log_end_time'],
      dtype='object')

In [36]:
feature_list = ['client_tx_bytes', 'service_tx_bytes','log_start_time', 'log_end_time']

In [37]:
def normalize_data(df):
    
    """ Method for normalizing data using zscore
    
    Args:
        df (dataframe)
        
    Attributes:
        df (dataframe): original dataframe with selected rows and columns
        X (dataframe): consists of copy of df for processing
    
    Returns:
        X(dataframe): Normalized dataframe
        
    """
    
    X = df
    for col in X:
        X[col] = stats.zscore(X[col])
        
    return X

In [38]:
df = select_agent(df, agent_id) # select dataframe for selected agent
df = select_service(df,service_id)
df_created_at = df['createdAt']
df = select_features(df, feature_list)
df = normalize_data(df)

In [39]:
df

,client_tx_bytes,service_tx_bytes,log_start_time,log_end_time
2,0.225271,0.021949,-1.722539,-1.721753
19,1.349201,1.768884,-1.713605,-1.711754
55,-0.727070,-0.680420,-1.695381,-1.693533
69,-0.325439,-0.472756,-1.685589,-1.683739
88,1.934453,1.994485,-1.676476,-1.674628
111,-0.790882,-0.792612,-1.667365,-1.665518
213,1.679516,1.859850,-1.522589,-1.520926
234,-0.466030,-0.274068,-1.512954,-1.511112
267,-0.640385,-0.594198,-1.501629,-1.499787
284,-0.652452,-0.521576,-1.491599,-1.492243


In [43]:
X = df.to_numpy()

In [44]:
X.shape

(103, 4)

In [63]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses

In [56]:
X = X.reshape(103,2,2,1)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, X, test_size=0.2, random_state=42)

In [71]:
latent_dim = 64 

class Autoencoder(tf.keras.Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(4, activation='sigmoid'),
      layers.Reshape((2, 2))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

In [72]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [73]:
autoencoder.fit(X_train, y_train,
                epochs=10,
                shuffle=True,
                validation_data=(X_test, y_test))

Train on 82 samples, validate on 21 samples
Epoch 1/10
82/82 [==============================] - 1s 6ms/sample - loss: 1.2788 - val_loss: 1.1614
Epoch 2/10
82/82 [==============================] - 0s 305us/sample - loss: 1.2588 - val_loss: 1.1430
Epoch 3/10
82/82 [==============================] - 0s 288us/sample - loss: 1.2382 - val_loss: 1.1247
Epoch 4/10
82/82 [==============================] - 0s 321us/sample - loss: 1.2181 - val_loss: 1.1070
Epoch 5/10
82/82 [==============================] - 0s 285us/sample - loss: 1.1976 - val_loss: 1.0892
Epoch 6/10
82/82 [==============================] - 0s 341us/sample - loss: 1.1792 - val_loss: 1.0710
Epoch 7/10
82/82 [==============================] - 0s 309us/sample - loss: 1.1597 - val_loss: 1.0529
Epoch 8/10
82/82 [==============================] - 0s 310us/sample - loss: 1.1400 - val_loss: 1.0347
Epoch 9/10
82/82 [==============================] - 0s 342us/sample - loss: 1.1216 - val_loss: 1.0166
Epoch 10/10
82/82 [=====================

In [77]:
autoencoder.predict(X_test)[0]

array([[0.44517893, 0.4449534 ],
       [0.38106978, 0.4466553 ]], dtype=float32)

In [76]:
X_test[0]

array([[[ 0.89727765],
        [ 0.27033902]],

       [[-0.6053585 ],
        [-0.60981902]]])